# step 0

- Ebrains https://ebrains.eu/ and the The Human Brain Project https://www.humanbrainproject.eu/en/

- Neuromorphic computing resources: SpiNNaker and BrainScaleS

- login to https://spinn-20.cs.man.ac.uk/hub/home

# info about SpiNNaker

- SpiNNaker the hardware http://apt.cs.manchester.ac.uk/projects/SpiNNaker/

- sPyNNaker the software https://www.frontiersin.org/articles/10.3389/fnins.2018.00816/full
    - doc https://spynnaker.readthedocs.io/_/downloads/en/latest/pdf/

- SpiNNaker mailing list google https://groups.google.com/g/spinnakerusers?pli=1

- https://www.youtube.com/watch?v=V3MlOAru6Qk

- comparison https://www.frontiersin.org/articles/10.3389/fnins.2018.00291/full

# info about PyNN

- PyNN 
    - paper https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2634533/
    - PyNN website http://neuralensemble.org/docs/PyNN/index.html
    - PyNN mailing list google

# goal of this tutorial on PyNN

- by using PyNN software on the SpiNNaker hardware, 

- you will know the basic scripts to design, run and analyze a spiking neuronal network (SNN) implemented on a neuromorphic system

- and get you ready to explore or extend the models

- content:
    - building the network
        - neurons
            - cell types
            - populations
            - recording variables
            
        - connections
            - synapse types
            - connections types
            - projections
    
    - running the network
        - example of an entry level SNN
        
    - analysis of the results 
        - some visualization and statistics
        
    - next steps: SNN model A and B
        

# import the simulator

In [ ]:
import pyNN.spiNNaker as sim
#import pyNN.nest as sim
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
# note on PyNN with nest

# how to install PyNN and nest

# neuron, brian, etc put link

In [ ]:
# sim + tab key to explore PyNN features (classes, modules, functions, instances)

# have a try...

# setup the simulator

In [ ]:
sim.setup(
    timestep=1, # [ms]
    min_delay=1, # [ms]
    max_delay=100) # [ms]

time_bins = 500 #ms

# the cell types 

- cell models and source models
- point neuron (aka mono-compartment model)
- http://neuralensemble.org/docs/PyNN/standardmodels.html

In [ ]:
sim.list_standard_models()

- e.g., with IF_cond_exp 
- Leaky integrate and fire model with fixed threshold and decaying-exponential post-synaptic conductance.
- explicit model https://arxiv.org/pdf/2003.13365.pdf

In [ ]:
# parameter names

sim.IF_cond_exp.get_parameter_names()

In [ ]:
# default parameters

standard_pars = sim.IF_cond_exp.default_parameters

print(sim.IF_cond_exp.default_parameters)

In [ ]:
# dictonary of testing parameters

testing_pars = {'tau_m': 20, 
                'cm': 1.0, 
                'v_rest': -65.0, 
                'v_reset': -70.0, 
                'v_thresh': -48.0, 
                'tau_syn_E': 5.0, 
                'tau_syn_I': 5.0, 
                'tau_refrac': 2, 
                'i_offset': 0.0, 
                'e_rev_E': 0.0, 
                'e_rev_I': -70.0}

In [ ]:
# initial values

sim.IF_cond_exp().default_initial_values

# making cell populations


In [ ]:
pops = {}

n_cells = {}
neuron_Model = {}
neuron_ModelPars = {}
initial_Values = {}

In [ ]:
# define a popolation of excitatory neurons in 1D

popName = 'Exc'
n_cells[popName] = 100
neuron_Model[popName] = sim.IF_cond_exp
neuron_ModelPars[popName] = testing_pars # sim.IF_cond_exp.default_parameters # or standard_pars or testing_pars
initial_Values[popName] = sim.IF_cond_exp.default_initial_values

pops[popName] = sim.Population(n_cells[popName], 
                               neuron_Model[popName],
                               neuron_ModelPars[popName],
                               label = popName)
                               # add spatial constraint

pops[popName].record(['spikes','v','gsyn_exc','gsyn_inh'])   


In [ ]:
# define a popolation of inhibitory neurons in 1D

popName = 'Inh'
n_cells[popName] = 100
neuron_Model[popName] = sim.IF_cond_exp
neuron_ModelPars[popName] = sim.IF_cond_exp.default_parameters # or standard_pars or testing_pars
initial_Values[popName] = sim.IF_cond_exp.default_initial_values

pops[popName] = sim.Population(n_cells[popName], 
                              neuron_Model[popName],
                              neuron_ModelPars[popName],
                              label = popName)
                              # add spatial constraint

pops[popName].record(['spikes','v','gsyn_exc','gsyn_inh'])


In [ ]:
# check created populations 

pops
#pops.keys()
#pops.items()
#pops.values()

In [ ]:
# get initial values
pops['Exc'].get_initial_values()

In [ ]:
pops['Exc'].get_initial_value('v') # voltage

In [ ]:
pops['Exc'].get_initial_value('isyn_exc') # excitatory conductance

In [ ]:
pops['Exc'].get_initial_value('isyn_inh') # inhibitory conductance

In [ ]:
do_run = True
if do_run:
    # set initial values, e.g., voltage
    pops['Exc'].initialize(v = sim.RandomDistribution('normal', (-65.0, 2.0)))
    pops['Exc'].get_initial_value('v') # voltage

In [ ]:
# visual check

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1,2, figsize=(7,5))
axes_list = fig.axes

axes_list[0].hist(np.asarray(pops['Exc'].get_initial_value('v')),
                  bins = 50,
                  cumulative = True)

axes_list[0].set_title('CDF of voltage [mV]')

axes_list[1].hist(np.asarray(pops['Exc'].get_initial_value('v')),
                  density = True,
                  bins = 50,
                  cumulative = False)


axes_list[1].set_title('PDF of voltage [mV]')

# synapses types

In [ ]:
# static synapse

weights = np.random.normal(loc=0.04, scale=0.01)

static_synapse = sim.StaticSynapse(weight=weights, delay=0.5) # weights in [uS] and delay in [ms]


In [ ]:
# spike-timinig-dependend plasticity (STDP)

do_run = False
if do_run:

    stdp_synapse = sim.STDPMechanism(
                                     timing_dependence=None,
                                     weight_dependence=None,
                                     voltage_dependence=None,
                                     dendritic_delay_fraction=1.0,
                                     weight = 0.02, # initial value
                                     delay = "0.5 + 0.01*d", # distance based delay min_delay + f(distance)
                                     )


# connectors types


In [ ]:
# all to all connections

connector = sim.AllToAllConnector(allow_self_connections=True) # autapses=True

In [ ]:
# one to one connections

connector = sim.OneToOneConnector()

In [ ]:
# stocastic connections

prob = 0.25
prob = np.random.normal(loc=0.5, scale=0.1)
connector = sim.FixedProbabilityConnector(p_connect=prob)

In [ ]:
# fixed number of post synptic neurons randomly taken (from i to j)

connector = sim.FixedNumberPostConnector(n = 50, allow_self_connections=True)

In [ ]:
# fixed number of pre synptic neurons randomly taken (from j to i)

connector = sim.FixedNumberPreConnector(n = 50, allow_self_connections=True)

In [ ]:
# distance based probability connections

d_rule = "exp(-d)" # or d_expression = "d<5"
connector = sim.DistanceDependentProbabilityConnector(d_expression=d_rule)

In [ ]:
# small-world connections

do_run = False
if do_run:
    connector = sim.SmallWorldConnector(
                                    degree,
                                    rewiring,
                                    allow_self_connections=True,
                                    safe=True,
                                    verbose=False,
                                    n_connections=None,
                                    )


In [ ]:
# list based connections, i.e., [i, j, weight, delay]

# one-to-one like list
givenList = [[0, 0, 0.08, 1],
             [1, 1, 0.08, 1],
             [2, 2, 0.08, 1],
             [3, 3, 0.08, 1],
             [4, 4, 0.08, 1]]
             #...

sim.FromListConnector(conn_list=givenList)


# example of a functionS to compute a list for list based connector

In [ ]:
# e.g., list of Distance based Probability Connections with Distance based Delay (DPCDD)

In [ ]:
def compute_CS_list(n_cells_i, n_cells_j, weights, d_thresh, p_thresh, width): 
    v_c = 0.35 #m/s
    d0 = 1 #ms
    scale = 1
    
    connections = {}
    probabilities = {}
    distances = {}

    connections = []
    probabilities = []
    distances = []
    for pre in range(n_cells_i):
        for post in range(n_cells_j):
            d_ij = np.sqrt((pre - post)**2)
            delay = d0 + d_ij / v_c
            if d_ij > d_thresh: 
                distances.append(d_ij)
                p_ij = scale*np.exp(-0.5 * (d_ij**2/width**2))
                probabilities.append(p_ij)

                if p_ij > p_thresh:
                    connections.append([pre, post, weights, delay])#, [d_ij, p_ij]])

    return connections, distances, probabilities

In [ ]:
# operative example 

n_cell = {'excitatory': 100, 'inhibitory': 100}
weights = {'excitatory': 0.008, 'inhibitory': 0.008}
d_thresh = {'excitatory': 0, 'inhibitory': 2} 
p_thresh = {'excitatory': 0.1, 'inhibitory': 0.1}
width = {'excitatory': 1, 'inhibitory': 3}

# look particular cases of one-to-one, all-to-all

In [ ]:
connections = {}
distances = {}
probabilities = {}

for synapse_type in ['excitatory', 'inhibitory']:
    connections[synapse_type], distances[synapse_type], probabilities[synapse_type] = compute_CS_list(
                                                                                                        n_cell[synapse_type], 
                                                                                                        n_cell[synapse_type], 
                                                                                                        weights[synapse_type], 
                                                                                                        d_thresh[synapse_type], 
                                                                                                        p_thresh[synapse_type],
                                                                                                        width[synapse_type])

In [ ]:
# visual check
fig, ax = plt.subplots(1,2, figsize=(11,5))
fig.tight_layout(pad=3)
axes_list = fig.axes

axes_list[0].plot(np.asarray(connections['excitatory']).T[0], np.asarray(connections['excitatory']).T[1],'go', label='excitatory')
axes_list[0].plot(np.asarray(connections['inhibitory']).T[0], np.asarray(connections['inhibitory']).T[1],'r+', label='inhibitory')
axes_list[0].grid()
axes_list[0].legend()
axes_list[0].set_title('scatter plot of connections')
axes_list[0].set_xlabel('i cells')
axes_list[0].set_ylabel('j cells')
axes_list[0].set_xlim(20,40)
axes_list[0].set_ylim(20,40)


axes_list[1].plot(distances['excitatory'],probabilities['excitatory'],'g+')
axes_list[1].plot(distances['inhibitory'],probabilities['inhibitory'],'r+') 
axes_list[1].plot(np.arange(0, 50), 1*np.exp(-0.5 * (np.arange(0, 50)**2/width['excitatory']**2)), 'g:')
axes_list[1].plot(np.arange(0, 50), 1*np.exp(-0.5 * (np.arange(0, 50)**2/width['inhibitory']**2)), 'r:')
axes_list[1].grid()
axes_list[1].set_xlim(0,50)
axes_list[1].set_ylim(0,1)
axes_list[1].axhline(p_thresh['excitatory'], color='k', label='p_threshold', )
axes_list[1].set_title('probability as function of distance')
axes_list[1].set_xlabel('distance')
axes_list[1].set_ylabel('probability')
axes_list[1].legend()


# designing projections

In [ ]:
# make list from center-surround function

weight = {('Exc', 'Exc') : 0.08,
           ('Exc', 'Inh') : 0.08,
           ('Inh', 'Inh') : 0.08,
           ('Inh', 'Exc') : 0.8
          }

d_thresh = {('Exc', 'Exc'): 0, 
            ('Inh', 'Inh'): 2} 

p_thresh = {('Exc', 'Exc'): 0.1, 
            ('Inh', 'Inh'): 0.1}

width = {('Exc', 'Exc'): 1, 
         ('Inh', 'Inh'): 3}

connections = {}
connections['Exc', 'Exc'], _, _ = compute_CS_list(n_cells['Exc'], n_cells['Exc'], weight['Exc', 'Exc'], d_thresh['Exc', 'Exc'], p_thresh['Exc', 'Exc'], width['Exc', 'Exc'])
connections['Inh', 'Inh'], _, _ = compute_CS_list(n_cells['Inh'], n_cells['Inh'], weight['Inh', 'Inh'], d_thresh['Inh', 'Inh'], p_thresh['Inh', 'Inh'], width['Inh', 'Inh'])

In [ ]:
proj = {}


proj['Exc', 'Exc'] = sim.Projection(pops['Exc'], pops['Exc'],
                                   connector = sim.FromListConnector(connections['Exc', 'Exc']),
                                   #synapse_type = sim.StaticSynapse(weight=0.08, delay=1),
                                   receptor_type = 'excitatory',
                                    #space=<pyNN.space.Space object at 0x7ff8f25a2110>,
                                   label = 'exc-exc connections'
                                    )

proj['Exc', 'Inh'] = sim.Projection(pops['Exc'], pops['Inh'],
                                   connector = sim.OneToOneConnector(), 
                                   synapse_type = sim.StaticSynapse(weight=weight['Exc', 'Inh'], delay=1),
                                   receptor_type = 'excitatory',
                                    #space=<pyNN.space.Space object at 0x7ff8f25a2110>,
                                   label = 'exc-inh connections'
                                    )

proj['Inh', 'Exc'] = sim.Projection(pops['Inh'], pops['Inh'],
                                   connector = sim.FromListConnector(connections['Inh', 'Exc']),
                                   #synapse_type = sim.StaticSynapse(weight=0.08, delay=1),
                                   receptor_type = 'inhibitory',
                                    #space=<pyNN.space.Space object at 0x7ff8f25a2110>,
                                   label = 'inh-inh connections'
                                    )



In [1]:
# add this
# only after run
print(len(proj['Thalamus', pop].get(['source', 'target', 'weight', 'delay'], format='list')))

# expected
connections['Inh', 'Exc'][0:10]

NameError: name 'proj' is not defined

In [ ]:
# space and structure

# setting of the stimulus

In [7]:
popName = 'Thalamus'  

n_cells[popName] = n_cells['Exc']
spike_times = [[]]*n_cells[popName] #list of spike lists, where one spike list is related to one spike source

random_sources_idx = [np.random.randint(n_cells[popName]*0.45, n_cells[popName]*0.55) for i in range(n_cells[popName])]

for idx, sources in enumerate(random_sources_idx):
    spike_times[sources] = [np.random.normal(loc=10, scale=0.01) for n in range(5)]

spike_times
%matplotlib
a = plt.eventplot(spike_times)
plt.xlabel('[ms]')
plt.ylabel('Thalamic cells')
plt.title('Raster plot of input spike sources')

#spike_times

NameError: name 'n_cells' is not defined

In [ ]:
neuron_Model[popName] = sim.SpikeSourceArray(spike_times)

pops[popName] = sim.Population(n_cells[popName], 
                               neuron_Model[popName],
                               label = popName)
                              # add spatial constraint
    
proj['Thalamus', 'Exc'] = sim.Projection(pops['Thalamus'], pops['Exc'],
               connector = sim.OneToOneConnector(),
               synapse_type = sim.StaticSynapse(weight=0.08, delay=1),
               receptor_type = 'excitatory',
                #space=<pyNN.space.Space object at 0x7ff8f25a2110>,
               label = 'thalamus-exc connections'
                )

# run simulation

In [ ]:
sim.run(time_bins) # time_bins=500 [ms]

# wait some minuts...

# save results

In [1]:
outputs = {}

for layer in ['Exc', 'Inh']:
    
    # save on the notebook space
    outputs[layer] = pops[layer].get_data()
    
    #save on local machine
    for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
        pops[layer].write_data(str(layer) + '_' + str(recording) + '.pkl')
        

NameError: name 'pops' is not defined

# close simulation

In [ ]:
sim.end()

# recover results

In [ ]:
def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            #print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results

In [ ]:
# check results
results = recover_results(outputs)
results.keys()

# looking the results 

In [ ]:
# eventually import pkl file

do_run = False

if do_run:
    import pickle
    
    #e.g., 
    with open('Exc_v.pkl', 'rb') as f:
            data = pickle.load(f)


In [ ]:
# check the spikes

fig, axes = plt.subplots(2, 1)
fig.tight_layout(pad=5)
axes_list = fig.axes

for idx, value in enumerate(['Exc', 'Inh']):
    axes_list[idx].eventplot(results[value, 'spikes'])
    axes_list[idx].set_title('rasterplot of ' + str(value) + ' layer')
    axes_list[idx].set_xlabel('[ms]')
    axes_list[idx].set_ylabel('cells')


In [ ]:
# check the voltage

fig, axes = plt.subplots(2, 1)
fig.tight_layout(pad=3)

axes_list = fig.axes

for idx, value in enumerate(['Exc', 'Inh']):
    im = axes_list[idx].imshow(results[value, 'v'].T)
    axes_list[idx].set_title('voltage of ' + str(value) + ' layer')
    axes_list[idx].set_xlabel('time [ms]')
    axes_list[idx].set_ylabel('cells')
    fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[mV]')

In [ ]:
# check the conductances

for layer in ['Exc', 'Inh']:
    fig, axes = plt.subplots(2, 1)
    fig.tight_layout(pad=5)
    fig.suptitle(str(layer) + ' layer')
    axes_list = fig.axes
    
    for idx, gsyn in enumerate(['gsyn_exc', 'gsyn_inh']):
        im = axes_list[idx].imshow(results[layer, gsyn].T)
        axes_list[idx].set_title(str(gsyn))
        axes_list[idx].set_xlabel('time [ms]')
        axes_list[idx].set_ylabel('cells')
        fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[uS]')

# some excercises
- write an equivalent PyNN function, AllToAll etc
- find balance, change design projection slot 
- open ring, close ring
- compare cell models
- compute statistics, data analysis, frequency domain (FFT, HHT)
- take one example from the PyNN website built for nest/neuron and adapt to run on SpiNNaker
  http://neuralensemble.org/docs/PyNN/examples.html
- inject current topic

## Exercises
The following exercises are designed to enhance your understanding of PyNN.  You have the option of following the step-by-step instructions in the linked notebooks, or you can try to follow the instructions below each task (this will be harder).

 - [Task 1: A simple neural network (Easy)](task1.ipynb)
    - Create a network with a timestep of 1.0ms consisting of 2 SpikeSourceArray neurons, spiking at 0.0ms and 1.0ms respectively, connected to 2 LIF neurons with a one-to-one connector, using a weight of 5.0nA and a delay of 2.0ms.  Record the spikes and run for 10ms, plotting the spikes after the simulation has completed.
    - See the notebook for further extensions.
 - [Task 2: Synfire Chain (Moderate)](task2.ipynb)
    - Create a "synfire chain" network from a single Population of 100 neurons using a FromListConnector to connect each neuron to the next neuron in the Population, additionally looping back around from the last to the first.  Use a SpikeSourceArray to stimulate the first neuron in the Population at the start.  Record and plot the spikes after running for 2 seconds.
    - See the notebook for further extensions.
 - [Task 3: Balanced Random Cortex-like Network (Hard)](task3.ipynb)
    - Create a "balanced random network" with a timestep of 0.1ms consisting of one "excitatory" and one "inhibitory" Population of LIF neurons with a size in the ratio of 4:1, and stimulate each in a one-to-one fashion using a SpikeSourcePoisson with rate 1000Hz.  Make the excitatory Population send excitatory spikes to itself and the inhibitory Population, and the inhibitory Population send inhibitory spikes to itself and the excitatory population.  Initialize the voltages using a random uniform distribution between -65.0 and -55.0.  Record and plot the spikes from the excitatory Population.
    - See the notebook for further extensions.
 - [Task 4: STDP Network (Moderate)](task4.ipynb)
     - Create a network of two single-neuron LIF Populations connected by an STDP connection, which has a tau_plus larger than tau_minus.  Stimulate each of the Populations repeatedly with gaps between stimulations, the first just before the second.  Plot the spikes of each.
     - See the notebook for further extensions.
 - [Task 5: STDP Curve (Hard)](task5.ipynb)
     - Create an STDP curve graph using a SpikeSourceArray stimulating a LIF population, with varying spike times for the SpikeSourceArray.
     - See the notebook for further extensions.


# receipt to design your own network
    - thalamic input
    - sim time and dt
    - layers
    - dimensions
    - connections types
    - cell types
    - synaptic types
    - weigth optimization
    - recordings
    - visualize and analysis

# FAQs